In [1]:
from Librerias_SAP import SAP_GUI,Funtions
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta

def Clean_Columns(List):
    Set_Columns=[i.strip() for i in List.columns]   #Alert in list
    clean_column=[i if "Unnamed" in i else None for i in List.columns]
    clean_column = list(filter(lambda x: x is not None, clean_column)) 
    List=List.set_axis(Set_Columns, axis=1).drop(clean_column,axis=1)  #Drop Unammed: 0
    return(List)

"""--------------------------------FUNCION PARA LIMPIAR NUMEROS A FLOAT--------------------------------------------------------"""

def Clean_column_number(column):
    column=column.apply(lambda x: float(str(x).strip().replace(",","")))
    return(column)

def default_column(default_columns,dataframe):     #Parametros (default_columns: Columnas predeterminadas,dataframe:)
    diccionay_default_column={}
    if len(default_columns)==len(dataframe.columns):
        for i in range(0,len(default_columns)):
            diccionay_default_column[dataframe.columns[i]]=default_columns[i]   
        dataframe=dataframe.rename(columns=diccionay_default_column)   #Remplazo las columnas con las de default
    else:
        None   #asumo columnas originales como estandar
    return(dataframe)

def Date_Null(dataframe,date):
    filter=(date!="00.00.0000")
    dataframe=dataframe.loc[filter]
    return(dataframe)

#Limpiar colummna a numeros
def Clean_Num_List(*args):
    lista=[]
    for arg in args:
        args=arg.apply(lambda x: '{:.3f}'.format(float(str(x).strip().replace(',',''))) if str(x).strip()!="0" else int(str(x).strip().replace(',','')))
        lista.append(args)
    if len(lista)==1:
        return lista[0]
    else:
        return(tuple(lista))




"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][1]
password=Keys["Password"][1]
# Initialize session
Iniciar=SAP_GUI.SessionSAP(user,password)

# Start transsación
Transsacion="mb52"
provision="GRANELES"   
variant="AUXNVOSPT"
Buscar_MB52_GRANELES=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)


Name_GRANELES="GRANELES"  # Name file
SAP_GUI.Export_TXT2(Name_GRANELES,Iniciar)

Ruta_GRANELES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_GRANELES)+".txt"
data_GRANELES = pd.read_csv(Ruta_GRANELES, skiprows=1, delimiter='\t')

data_GRANELES=Clean_Columns(data_GRANELES)

default_column_MB52=['Material', 'Texto breve de material', 'Gpo.artíc.', 'Ce.', 'Alm.',
       'Lote', 'UMB', 'Libre utilización', 'En control calidad',
       'Valor libre util.', 'Valor en insp.cal.', 'Bloqueado',
       'Stock no libre', 'Valor stock bloq.', 'Valor no libre', 'Cad./FPC'] #Disposicion PROXI A VENCER

data_GRANELES=default_column(default_column_MB52,data_GRANELES) 


data_GRANELES=Date_Null(data_GRANELES,data_GRANELES['Cad./FPC'])

try:
    data_GRANELES["Alm."]=data_GRANELES["Alm."].astype(int)
    data_GRANELES["Lote"]=data_GRANELES["Lote"].astype(int)
    data_GRANELES['Ce.']=data_GRANELES['Ce.'].astype(int)
except:
    None



"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_GRANELES=data_GRANELES[data_GRANELES.Material.notnull()]
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].apply(lambda x: datetime.strptime(x,'%d.%m.%Y'))  #Change columns str to datetime

Data_Denominacion=Clean_Columns(pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Denominacion_Articulos.txt",delimiter='\t'))[["Grupo art.","Denominación 2 del gr.artículos"]]
Data_Denominacion=Data_Denominacion.rename(columns={"Denominación 2 del gr.artículos":"Denom.gr.artíc."})
data_GRANELES=pd.merge(Data_Denominacion,data_GRANELES,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")


data_GRANELES['Libre utilización'],data_GRANELES['En control calidad'],data_GRANELES['Valor libre util.'],data_GRANELES['Valor en insp.cal.'],data_GRANELES['Bloqueado'],data_GRANELES['Stock no libre'],data_GRANELES['Valor stock bloq.'],data_GRANELES['Valor no libre']=Clean_Num_List(data_GRANELES['Libre utilización'],data_GRANELES['En control calidad'],data_GRANELES['Valor libre util.'],data_GRANELES['Valor en insp.cal.'],data_GRANELES['Bloqueado'],data_GRANELES['Stock no libre'],data_GRANELES['Valor stock bloq.'],data_GRANELES['Valor no libre'])

"""------------------Considero los datos de junio----------------------------------------------------------------------"""
####
###
#
data_Precios_Valores_Junio=pd.read_excel("C:\\Users\\prac.ingindustrial2\\Downloads\\Precios_Valores_Junio.XLSX")

data_GRANELES=pd.merge(data_GRANELES,data_Precios_Valores_Junio,on="Material")

data_GRANELES['Valor no libre'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor no libre']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor no libre']!=0 else row['Valor no libre'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_GRANELES['Valor libre util.'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor libre util.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor libre util.']!=0 else row['Valor libre util.'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_GRANELES['Valor stock bloq.'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor stock bloq.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor stock bloq.']!=0 else row['Valor stock bloq.'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_GRANELES['Valor en insp.cal.'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor en insp.cal.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor en insp.cal.']!=0 else row['Valor en insp.cal.'],axis=1)
#
##
###
"""------------------Considero los datos de junio----------------------------------------------------------------------"""


"Transfromo los datos a numericos"

"""---------------------------------Filtro graneles vencidos-------------------"""
now=datetime.now()
month_vencido=now-timedelta(days=30) 
mask_vencido=(data_GRANELES["Cad./FPC"]<=now) & (data_GRANELES["Cad./FPC"]>=month_vencido)
data_GRANELES_vencidos=data_GRANELES.loc[mask_vencido]
#Sort values
data_GRANELES_vencidos=data_GRANELES_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_GRANELES_vencidos=data_GRANELES_vencidos.reset_index(drop=True)
data_GRANELES_vencidos["Cad./FPC"]=data_GRANELES_vencidos["Cad./FPC"].dt.date   #convet datetime to date 



# filter dates between now a 30 month later
now=datetime.now()
month=timedelta(days=30)   # Change limiter day(30 days= 1 month)
month=now+month
mask=(data_GRANELES["Cad./FPC"]>=now) & (data_GRANELES["Cad./FPC"]<=month)
data_GRANELES=data_GRANELES.loc[mask]
#Sort values
data_GRANELES=data_GRANELES.sort_values(by=["Cad./FPC","Libre utilización"])
data_GRANELES=data_GRANELES.reset_index(drop=True)
data_GRANELES["Cad./FPC"]=data_GRANELES["Cad./FPC"].dt.date   #convet datetime to date 


In [ ]:
data_GRANELES['Valor no libre'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor no libre']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor no libre']!=0 else row['Valor no libre'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_GRANELES['Valor libre util.'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor libre util.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor libre util.']!=0 else row['Valor libre util.'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_GRANELES['Valor stock bloq.'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor stock bloq.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor stock bloq.']!=0 else row['Valor stock bloq.'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_GRANELES['Valor en insp.cal.'] = data_GRANELES.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor en insp.cal.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor en insp.cal.']!=0 else row['Valor en insp.cal.'],axis=1)
#

In [2]:
"""------------DATAFRAME GRANALES PARA INFORME-----------------------------------------------------------------------"""

data_Graneles_informe=data_GRANELES[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]


data_Graneles_Vencidos_informe=data_GRANELES[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote','Bloqueado','Stock no libre', 'UMB','Valor stock bloq.', 'Valor no libre',
       'Cad./FPC']]



In [3]:
"""-----------------------------GENERO SUBTOTALES DE GRANELES------------------------------------------------------------"""

data_GRANELES_Libre=Clean_column_number(data_GRANELES["Valor libre util."]).sum()
data_GRANELES_Calidad=Clean_column_number(data_GRANELES['Valor en insp.cal.']).sum()
Total_Graneles=data_GRANELES_Libre+data_GRANELES_Calidad
Total_Graneles="${:,.2f}".format(Total_Graneles)


"""-----------------------------GENERO SUBTOTALES DE GRANELES VENCIDOS--------------------------------------------------------"""
data_GRANELES_Stock_Bloqueado=Clean_column_number(data_GRANELES_vencidos['Valor stock bloq.']).sum()
data_GRANELES_No_Libre=Clean_column_number(data_GRANELES_vencidos['Valor no libre']).sum()
Total_Graneles_Vencidos=data_GRANELES_Stock_Bloqueado+data_GRANELES_No_Libre
Total_Graneles_Vencidos="${:,.2f}".format(Total_Graneles_Vencidos)


In [4]:
""" Search MB52 raw material"""

# Start transsación
Transsacion="mb52"
provision="MATERIA PRIMA"  
variant="AUXNVOSPT"
Buscar_MB52_MATERIAS=SAP_GUI.Search_MB52(Transsacion,Iniciar,provision,variant)
# Save file



Name_Materias="MATERIAS"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_Materias,Iniciar)
# Pass the route and read file 
Ruta_Materias="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_Materias)+".txt"
data_Materias=pd.read_csv(Ruta_Materias,delimiter="\t")
# Change, drop and standardize columns
data_Materias=Clean_Columns(data_Materias)
data_Materias=default_column(default_column_MB52,data_Materias) 
data_Materias=Date_Null(data_Materias,data_Materias['Cad./FPC'])

try:
    # Change Float-Int
    data_Materias["Alm."]=data_Materias["Alm."].astype(int)
    data_Materias["Lote"]=data_Materias["Lote"].astype(int)
    data_Materias['Ce.']=data_Materias['Ce.'].astype(int)
except:
    None


data_Materias['Libre utilización'],data_Materias['En control calidad'],data_Materias['Valor libre util.'],data_Materias['Valor en insp.cal.'],data_Materias['Bloqueado'],data_Materias['Stock no libre'],data_Materias['Valor stock bloq.'],data_Materias['Valor no libre']=Clean_Num_List(data_Materias['Libre utilización'],data_Materias['En control calidad'],data_Materias['Valor libre util.'],data_Materias['Valor en insp.cal.'],data_Materias['Bloqueado'],data_Materias['Stock no libre'],data_Materias['Valor stock bloq.'],data_Materias['Valor no libre'])

"""------------------Considero los datos de junio----------------------------------------------------------------------"""

"""-------------------------Filtro para eliminar subtotales y combertir fechas-------------------------------"""
data_Materias=data_Materias[data_Materias.Material.notnull()]
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].apply(lambda x: datetime.strptime(str(x),'%d.%m.%Y'))  #Change columns str to datetime

data_Materias=pd.merge(Data_Denominacion,data_Materias,how="right",left_on="Grupo art.",right_on="Gpo.artíc.")


"""---------------------------Buscamos las clases de material------------- """
Data_Material_Total=data_Materias
Lista_tipo_Materias=data_Materias["Gpo.artíc."].drop_duplicates().reset_index(drop=True)


"""------------------Considero los datos de junio----------------------------------------------------------------------"""
####
###
#
data_Materias=pd.merge(data_Materias,data_Precios_Valores_Junio,on="Material")

data_Materias['Valor no libre'] = data_Materias.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor no libre']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor no libre']!=0 else row['Valor no libre'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_Materias['Valor libre util.'] = data_Materias.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor libre util.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor libre util.']!=0 else row['Valor libre util.'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_Materias['Valor stock bloq.'] = data_Materias.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor stock bloq.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor stock bloq.']!=0 else row['Valor stock bloq.'],axis=1)
# Aplicar una función lambda a múltiples columnas
data_Materias['Valor en insp.cal.'] = data_Materias.apply(lambda row: row["Precio interno periódico Estimado"] if row['Valor en insp.cal.']!=0 and row["Precio interno periódico Estimado"]!=0 else row['Precio estándar'] if row["Precio interno periódico Estimado"]==0 and row['Valor en insp.cal.']!=0 else row['Valor en insp.cal.'],axis=1)
#
##
###

In [8]:
data_Materias

,Grupo art.,Denom.gr.artíc.,Material,Texto breve de material,Gpo.artíc.,Ce.,Alm.,Lote,UMB,Libre utilización,...,Valor en insp.cal.,Bloqueado,Stock no libre,Valor stock bloq.,Valor no libre,Cad./FPC,Precio interno periódico,Grupo de artículos,Precio estándar,Precio interno periódico Estimado
0,C011,MP. Prebel,PB0062306,CAPRYLYL CAPRYLATE/CAPRATE,C011,1000.0,1001.0,10878493.0,KG,2.500,...,0,0,0,0,0,2023-07-07,92102637,C011,92102637,92102637
1,C274,MP.BEIERSDORF,PB0084145,LICORICE EXTRACT GW/00199,C274,1000.0,1001.0,10839215.0,KG,4.770,...,0,1.000,0,232235779,0,2023-07-07,232235779,C274,232235779,232235779
2,C011,MP. Prebel,0018625600,CMC 7H3SF AQUALON,C011,1000.0,1001.0,10656400.0,G,2385.500,...,0,0,0,0,0,2023-07-08,168984,C011,168984,168984
3,C011,MP. Prebel,0DY0601200,POLYMETHYL METHACRYLATE,C011,1000.0,1001.0,10848463.0,G,34750.000,...,0,0,0,0,0,2023-07-08,219483,C011,214279,219483
4,C274,MP.BEIERSDORF,PB0085740,SEQUESTRANT/7620,C274,1000.0,1001.0,10816289.0,KG,24.119,...,0,0,0,0,0,2023-07-09,36389897,C274,36389897,36389897
5,C011,MP. Prebel,PB0069334,YELLOW AQUA MIYOSHI,C011,1000.0,1001.0,10823709.0,KG,4.387,...,0,0,0,0,0,2023-07-09,180125962,C011,180126197,180125962
6,C242,MP.HENKEL,PB0086250,PROMOIS SILK-A,C242,1000.0,1001.0,10864547.0,KG,14.889,...,0,0,0,0,0,2023-07-10,1990754584,C242,1990754584,1990754584
7,C011,MP. Prebel,PB0018460,YOGURTENE CQ (PFR) U-A IO,C011,1000.0,1001.0,10873129.0,KG,21.089,...,0,0,0,0,0,2023-07-10,183888374,C011,183888352,183888374
8,C272,MP.LOCATEL,PB0083217,FRAG.FRESH SENSATION AR680580,C272,1000.0,1001.0,10871505.0,KG,4.910,...,0,0,0,0,0,2023-07-10,99202087,C272,99202087,99202087
9,C011,MP. Prebel,PB0047773,COLIPA POWDERSHEEN 11A RQ-1242 PF,C011,1000.0,1001.0,10862546.0,KG,0.300,...,0,0,0,0,0,2023-07-11,3333,C011,948,3333


In [5]:
"""--------------------------------------MATERIAS PRIMAS VENCIDAS-------------------------"""

mask_vencido=(data_Materias["Cad./FPC"]<=now) & (data_Materias["Cad./FPC"]>=month_vencido)
data_Materias_vencidos=data_Materias.loc[mask_vencido]
#Sort values
data_Materias_vencidos=data_Materias_vencidos.sort_values(by=["Cad./FPC"],ascending=False)
data_Materias_vencidos=data_Materias_vencidos.reset_index(drop=True)
data_Materias_vencidos["Cad./FPC"]=data_Materias_vencidos["Cad./FPC"].dt.date   #convet datetime to date 


mask=(data_Materias["Cad./FPC"]>=now) & (data_Materias["Cad./FPC"]<=month)
data_Materias=data_Materias.loc[mask]
data_Materias=data_Materias.sort_values(by=["Cad./FPC","Libre utilización"]).reset_index(drop=True)
data_Materias["Cad./FPC"]=data_Materias["Cad./FPC"].dt.date

#Close Session

In [6]:
x=dsf

NameError: name 'dsf' is not defined

In [ ]:
#SAP_GUI.Close_session(Iniciar)


"""------------DATAFRAME MATERIAS PARA INFORME-----------------------------------------------------------------------"""
data_Materia_informe=data_Materias[['Material', 'Texto breve de material', 'Gpo.artíc.','Denom.gr.artíc.', 'Ce.', 'Alm.',
       'Lote',  'Libre utilización', 'En control calidad','UMB', 'Valor libre util.', 'Valor en insp.cal.','Cad./FPC']]

data_Materia_informe_vencidos=data_Materias_vencidos[['Grupo art.', 'Denom.gr.artíc.', 'Material', 'Texto breve de material',
       'Gpo.artíc.', 'Ce.', 'Alm.', 'Lote','Bloqueado','Stock no libre', 'UMB','Valor stock bloq.', 'Valor no libre',
       'Cad./FPC']]

"""-----------------------------GENERO SUBTOTALES MATERIAS PRIMAS------------------------------------------------------------"""

data_Materia_Libre=Clean_column_number(data_Materias["Valor libre util."]).sum()
data_Materia_Calidad=Clean_column_number(data_Materias['Valor en insp.cal.']).sum()
Total_Materia=data_Materia_Libre+data_Materia_Calidad
Total_Materia="${:,.2f}".format(Total_Materia)


"""-----------------------------GENERO SUBTOTALES DE MATERIAS PRIMAS VENCIDAS--------------------------------------------------------"""
data_Materia_Stock_Bloqueado=Clean_column_number(data_Materias_vencidos['Valor stock bloq.']).sum()
data_Materia_No_Libre=Clean_column_number(data_Materias_vencidos['Valor no libre']).sum()
Total_Materia_Vencidos=data_Materia_Stock_Bloqueado+data_Materia_No_Libre
Total_Materia_Vencidos="${:,.2f}".format(Total_Materia_Vencidos)

"""Send email"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \
        .highlight_between(subset=["Cad./FPC"],color='#FF5733',left=now.date(),right=(now.date()+timedelta(days=7))) \

html="""
    <h2 style="text-align: center">REPORTE CADUCIDAD DE GRANELES</h2>
    <p> Por medio del presente informe se evidencia los graneles segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>

    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Los graneles a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

html2="""
    <h2 style="text-align: center"> REPORTE CADUCIDAD DE MATERIAS PRIMAS</h2>
    <p> Por medio del presente informe se evidencia las materias primas segun el tipo de material que estan proximos a vencer durante un mes a partir de la 
        fecha actual, ademas de las cantidades con su respectiva unidad de medida que estan en libre utilización.</p>

    <h4 style="color: black;" > Advertencia: El valor económico potencial en riesgo de vencimiento es de: </h4>
    
    <h1 style="color: red;" > {1} </h1>
    
    <div"> {0} </div>

    <h4 style="color: red;" > Advertencia: Las materias primas proximas a vencer en los proximos 7 días se resaltan en rojo</h4>

    <p> Anticipo sinceros agradecimientos. </p>
 """

Send = style_df(data_Graneles_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe)     #Style between LI and LS



# Definir el diccionario de formato
try:
    formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}
    formato_material= {'Ce.': '{:.0f}',"Alm.":'{:.0f}',"Lote":'{:.0f}'}
    # Aplicar el formato a la columna 'Altura'
    Send = Send.format(formato_Graneles)
    Send2=Send2.format(formato_material)
except:
    None

send_emails(Send.to_html(),Total_Graneles,emails=correos,htmlbody=html,subject="REPORTE GRANELES PROXIMOS A VENCER")
send_emails(Send2.to_html(),Total_Materia,emails=correos,htmlbody=html2,subject="REPORTE MATERIAS PRIMAS PROXIMOS A VENCER")


"""-------------------------------------Send email material y granel vencidos--------------------------------"""

correos="prac.ingindustrial2@prebel.com.co"

def send_emails(*args,emails="",htmlbody="",subject=""):
    email=emails
    outlook=win32com.client.Dispatch("outlook.application")
    mail=outlook.CreateItem(0)
    mail.Subject=subject+" "+datetime.now().strftime('%#d %b %Y %H:%M')
    mail.To=email
    mail.HTMLBody=htmlbody.format(*args)
    mail.Send()

def style_df(df):
    return df.style \
        .set_table_styles([{'selector': "table,tr,th,td", 'props': [("border", "1px solid"), ('color', '#000'),("text-align","center")]}]) \

html="""
    <h2 style="text-align: center">"REPORTE GRANELES Y MARIA PRIMA VENCIDO"</h2>
    <p> Por medio del presente informe se evidencia los graneles y materias primas vencidas, para su correspondiente análisis. </p>

    <h2 style="color: black;" > GRANELES VENCIDOS</h4>

    <h1 style="color: red;" > {2} </h1>
    <div"> {0} </div>

    <h2 style="color: black;" > MATERIA PRIMA VENCIDA </h4>
    
    <h1 style="color: red;" > {3} </h1>
    <div"> {1} </div>


    <h4> Anticipo sinceros agradecimientos. </h4>
 """


Send = style_df(data_Graneles_Vencidos_informe)  #Style between LI and LS
Send2=style_df(data_Materia_informe_vencidos)     #Style between LI and LS

try:
    formato_Graneles = {'Ce.': '{:.0f}',"Alm.":'{:.0f}'}
    formato_material= {'Ce.': '{:.0f}',"Alm.":'{:.0f}',"Lote":'{:.0f}'}
    # Aplicar el formato a la columna 'Altura'
    Send = Send.format(formato_Graneles)
    Send2=Send2.format(formato_material)
except:
    None

send_emails(Send.to_html(),Send2.to_html(),Total_Graneles_Vencidos,Total_Materia_Vencidos,emails=correos,htmlbody=html,subject="REPORTE GRANELES Y MARIA PRIMA VENCIDO")
